### SECTION C- Task Detail: Descriptive Statistics

#### Load Data

In [12]:
import pandas as pd
import sqlite3

# Connect to database
# conn = sqlite3.connect('queue_system_sqlite3.db')
conn = sqlite3.connect('cleaned_database.db')

#load data into dataframe
calls = pd.read_sql_query("SELECT * FROM calls", conn)
queues = pd.read_sql_query("SELECT * FROM queues", conn)

# Close the connection
conn.close()

#### Calculate Key metrics

In [13]:
# Merge calls and queues on queue_id to calculate metrics
calls_queues = pd.merge(calls, queues, left_on='queue_id', right_on='id', suffixes=('_call', '_queue'))

# Ensure datetime columns are in the correct format
calls_queues['created_at_call'] = pd.to_datetime(calls_queues['created_at_call'])
calls_queues['updated_at_call'] = pd.to_datetime(calls_queues['updated_at_call'])
calls_queues['created_at_queue'] = pd.to_datetime(calls_queues['created_at_queue'])

# Calculate waiting time, service time and turn-around time
calls_queues['waiting_time'] = (calls_queues['created_at_call'] - calls_queues['created_at_queue']).dt.total_seconds()
calls_queues['service_time'] = (calls_queues['updated_at_call'] - calls_queues['created_at_call']).dt.total_seconds()
calls_queues['turn_around_time'] = (calls_queues['updated_at_call'] - calls_queues['created_at_queue']).dt.total_seconds()

calls_queues[['queue_id', 'waiting_time', 'service_time', 'turn_around_time']].head()

,queue_id,waiting_time,service_time,turn_around_time
0,45,873.0,13.0,886.0
1,46,819.0,8.0,827.0
2,47,617.0,4.0,621.0
3,50,29.0,62.0,91.0
4,51,1997.0,6.0,2003.0


#### Calculate Statistics

In [14]:
# Calculate descriptive statistics for waiting time, service time, and turn-around time
descriptive_stats = {
    'waiting_time': {
        'mean': calls_queues['waiting_time'].mean(),
        'median': calls_queues['waiting_time'].median(),
        'std': calls_queues['waiting_time'].std()
    },
    'service_time': {
        'mean': calls_queues['service_time'].mean(),
        'median': calls_queues['service_time'].median(),
        'std': calls_queues['service_time'].std()
    },
    'turn_around_time': {
        'mean': calls_queues['turn_around_time'].mean(),
        'median': calls_queues['turn_around_time'].median(),
        'std': calls_queues['turn_around_time'].std()
    }
}

# Display the descriptive statistics
print("Descriptive Statistics:")
for metric, stats in descriptive_stats.items():
    print(f"\n{metric.capitalize()}:")
    print(f"  Mean: {stats['mean']}")
    print(f"  Median: {stats['median']}")
    print(f"  Standard Deviation: {stats['std']}")


Descriptive Statistics:

Waiting_time:
  Mean: 4174.621848739495
  Median: 3688.0000000000005
  Standard Deviation: 2952.797983092017

Service_time:
  Mean: 697.4468396054074
  Median: 637.0
  Standard Deviation: 635.9298685412648

Turn_around_time:
  Mean: 4872.068688344903
  Median: 4363.0
  Standard Deviation: 3066.2194361320758
